<a href="https://colab.research.google.com/github/Baekhyeonjeong/movie-review-/blob/main/%EC%98%81%ED%99%94_%EB%A7%90%ED%95%A0%EC%88%98%EC%97%86%EB%8A%94%EB%B9%84%EB%B0%80_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy 설치하기
!pip install konlpy

In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 말할수.csv
## https://drive.google.com/file/d/1nR8-y13SjtweE0FDc26z6HRQmtlIwq7a/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1nR8-y13SjtweE0FDc26z6HRQmtlIwq7a'})
rawdata_downloaded.GetContentFile('말할수.csv')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('말할수.csv', sep=",")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,와리천재(gusw****),"이게 진짜 명작이다.... 보기전, 대만영화라서 편견이 있었는데, 완전히 반성한다....",10,2443,117,2013-07-05 21:59:00,0
1,1,바나나(8745****),몇번봐도 너무 재미씀,10,1725,75,2013-06-15 12:35:00,0
2,2,별(lya9****),이런 영화 다시 안나오나..?,10,1566,64,2013-07-06 14:00:00,0
3,3,김민영(myki****),너무너무좋아...,10,1177,60,2013-07-24 21:59:00,0
4,4,하냥(chlt****),영화 전문가들은 대체 어느영화에 10점을 줄지 궁금하다,10,1069,65,2013-07-21 21:44:00,0
...,...,...,...,...,...,...,...,...
37725,37725,레미파라(play****),음악잘고르면 뭐해~영화가 쓰레긴데,1,10,24,2013-10-11 11:48:00,0
37726,37726,응억이(sehi****),이게 정녕 재미있는 영화인가? 따분한 러브스토리 전개에 예상할수 있는 결말. 기억에...,1,13,27,2014-04-06 01:06:00,0
37727,37727,시네마(kpar****),누가 평점 조작하냐..,1,11,27,2014-07-15 15:20:00,0
37728,37728,망고스틴(sonn****),재미있긴한데 평점이 너무 높다.,1,12,28,2014-09-09 00:43:00,0


In [ ]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [ ]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,와리천재(gusw****),"이게 진짜 명작이다.... 보기전, 대만영화라서 편견이 있었는데, 완전히 반성한다....",10,2443,117,2013-07-05 21:59:00,0
1,1,바나나(8745****),몇번봐도 너무 재미씀,10,1725,75,2013-06-15 12:35:00,0
2,2,별(lya9****),이런 영화 다시 안나오나..?,10,1566,64,2013-07-06 14:00:00,0
3,3,김민영(myki****),너무너무좋아...,10,1177,60,2013-07-24 21:59:00,0
4,4,하냥(chlt****),영화 전문가들은 대체 어느영화에 10점을 줄지 궁금하다,10,1069,65,2013-07-21 21:44:00,0
...,...,...,...,...,...,...,...,...
37725,37725,레미파라(play****),음악잘고르면 뭐해~영화가 쓰레긴데,1,10,24,2013-10-11 11:48:00,0
37726,37726,응억이(sehi****),이게 정녕 재미있는 영화인가? 따분한 러브스토리 전개에 예상할수 있는 결말. 기억에...,1,13,27,2014-04-06 01:06:00,0
37727,37727,시네마(kpar****),누가 평점 조작하냐..,1,11,27,2014-07-15 15:20:00,0
37728,37728,망고스틴(sonn****),재미있긴한데 평점이 너무 높다.,1,12,28,2014-09-09 00:43:00,0


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('이런', 'Modifier'),
 ('식', 'Noun'),
 ('으로', 'Josa'),
 ('관람객', 'Noun'),
 ('을', 'Josa'),
 ('우롱', 'Noun'),
 ('하지', 'Verb'),
 ('마세요', 'Verb'),
 ('!', 'Punctuation'),
 ('작은', 'Adjective'),
 ('스크린', 'Noun'),
 ('에', 'Josa'),
 ('맞추느라', 'Verb'),
 ('화면', 'Noun'),
 ('다', 'Adverb'),
 ('짜르', 'Noun'),
 ('고', 'Josa'),
 (',', 'Punctuation'),
 ('영화', 'Noun'),
 ('도', 'Josa'),
 ('다', 'Adverb'),
 ('짤', 'Noun'),
 ('라', 'Josa'),
 ('제멋대로', 'Noun'),
 ('편집', 'Noun'),
 (',', 'Punctuation'),
 ('좋은', 'Adjective'),
 ('영화', 'Noun'),
 ('를', 'Josa'),
 ('다', 'Adverb'),
 ('망쳐놨네요', 'Verb'),
 ('!', 'Punctuation'),
 ('돈', 'Noun'),
 ('물러', 'Verb'),
 ('달라', 'Noun'),
 ('고', 'Josa'),
 ('하고', 'Verb'),
 ('싶지만', 'Verb'),
 ('참고', 'Noun'),
 (',', 'Punctuation'),
 ('앞', 'Noun'),
 ('으로', 'Josa'),
 ('아트', 'Noun'),
 ('나인', 'Noun'),
 ('에는', 'Josa'),
 ('안옵니다', 'Verb'),
 ('!', 'Punctuation')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,이런,Modifier,1,1
1,식,Noun,1,1
2,으로,Josa,1,1
3,관람객,Noun,1,1
4,을,Josa,1,1
5,우롱,Noun,1,1
6,하지,Verb,1,1
7,마세요,Verb,1,1
8,!,Punctuation,1,1
9,작은,Adjective,1,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0            1  type  count
 0      이         Noun    10      1
 1      게         Josa    10      1
 2     진짜         Noun    10      1
 3     명작         Noun    10      1
 4     이다         Josa    10      1
 5   ....  Punctuation    10      1
 6    보기전         Verb    10      1
 7      ,  Punctuation    10      1
 8      대     Modifier    10      1
 9      만     Modifier    10      1
 10    영화         Noun    10      1
 11    라서         Josa    10      1
 12    편견         Noun    10      1
 13     이         Josa    10      1
 14  있었는데    Adjective    10      1
 15     ,  Punctuation    10      1
 16   완전히    Adjective    10      1
 17    반성         Noun    10      1
 18     한         Josa    10      1
 19     다       Adverb    10      1
 20  ....  Punctuation    10      1,     0       1  type  count
 0  몇번    Noun    10      1
 1  봐도    Verb    10      1
 2  너무  Adverb    10      1
 3  재미    Noun    10      1
 4   씀    Verb    10      1,      0            1  type  count
 0   이

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,이,Noun,10,1
1,게,Josa,10,1
2,진짜,Noun,10,1
3,명작,Noun,10,1
4,이다,Josa,10,1
...,...,...,...,...
42,아트,Noun,1,1
43,나인,Noun,1,1
44,에는,Josa,1,1
45,안옵니다,Verb,1,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,이,Noun,10,1
1,게,Josa,10,1
2,진짜,Noun,10,1
3,명작,Noun,10,1
4,이다,Josa,10,1
...,...,...,...,...
42,아트,Noun,1,1
43,나인,Noun,1,1
44,에는,Josa,1,1
45,안옵니다,Verb,1,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소     품사           분류
       Foreign      10     1
\n      Foreign      10     2
\r\n    Foreign      10     4
!       Punctuation  1     28
                     2      2
                           ..
不能說     Foreign      8      1
不能說的    Foreign      10     1
不能說的秘密  Foreign      9      1
                     10     2
＊       Foreign      6      3
Name: 카운트, Length: 37484, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소    품사          분류     
      Foreign     10    1
\n     Foreign     10    2
\r\n   Foreign     10    4
!      Punctuation 1    28
                   2     2
...                    ...
不能說    Foreign     8     1
不能說的   Foreign     10    1
不能說的秘密 Foreign     9     1
                   10    2
＊      Foreign     6     3

[37484 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')